In [ ]:
!pip install evm_cfg_builder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.3/228.3 kB 13.6 MB/s eta 0:00:00
  Created wheel for evm_cfg_builder: filename=evm_cfg_builder-0.3.1-py3-none-any.whl size=1264526 sha256=fcc850a2013d17cdad313966c92d37cc5cacd6321f0f93ad7cc7e39bf2988df2
  Stored in directory: /root/.cache/pip/wheels/11/f6/b0/6602981654810381d55fed6183edeccb6cdc7e505dfe4fc109
Successfully built evm_cfg_builder


In [ ]:
!pip install gensim nltk

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.9 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from evm_cfg_builder.cfg import CFG
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import nltk
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import json
from torch_geometric.data import Data

model = Word2Vec.load("/content/drive/MyDrive/Colab_Notebooks/Datasets/word2vec_opcode_16.bin")

def map_opcode_to_embedding(opcode_sequence):
    tokens = word_tokenize(opcode_sequence)
    opcode_sequence = [token.upper() for token in tokens if token.isalnum()]
    embedding_sequence = [model.wv[token] for token in opcode_sequence]
    average_embedding = np.mean(embedding_sequence, axis=0)
    max_embedding = np.max(embedding_sequence, axis=0)
    sum_embedding = np.sum(embedding_sequence, axis=0)
    final_embedding = np.concatenate([average_embedding, max_embedding, sum_embedding])
    norm = np.linalg.norm(final_embedding)
    if norm > 0:
        final_embedding = final_embedding / norm
    final_embedding = np.resize(final_embedding, (48))
    return final_embedding


In [ ]:
# Specify the path to your JSON file
json_file_path = "/content/drive/MyDrive/Colab_Notebooks/Datasets/reentrancy_train_dataset.json"

# Load the JSON data into a Python dictionary
with open(json_file_path, 'r') as json_file:
    dataset = json.load(json_file)


In [ ]:
def process_block(block):
    block_opcode = [info[0] for info in block['info']]
    block_embedding = map_opcode_to_embedding('\n'.join(block_opcode))
    return block_embedding

def process_bytecode(bytecode):
    cfg = CFG(bytecode)
    sorted_data_mapping = {
        str(block): {
            'pos': block.start.pc,
            'info': [(instr.mnemonic, instr.description) for instr in block.instructions],
            'out': [str(out_block) for out_block in block.all_outgoing_basic_blocks],
        }
        for block in cfg.basic_blocks
    }
    sorted_keys = sorted(sorted_data_mapping.keys(), key=lambda key: sorted_data_mapping[key]['pos'])

    graph_embedding = [process_block(sorted_data_mapping[key]) for key in sorted_keys]

    x = torch.tensor(np.vstack(graph_embedding), dtype=torch.float)
    edge_index = torch.tensor([
        [sorted_data_mapping[key]['pos'] for key in sorted_keys for _ in sorted_data_mapping[key]['out']],
        [sorted_data_mapping[out_block]['pos'] for key in sorted_keys for out_block in sorted_data_mapping[key]['out']]
    ], dtype=torch.long)

    return Data(x=x, edge_index=edge_index)

In [ ]:
bytecodes = dataset['bytecode']
labels = dataset['is_reentrancy']

data_list = []

for bytecode, label in tqdm(zip(bytecodes, labels), desc="Processing Bytecodes"):
    try:
        if bytecode == "0x":
            continue
        data = process_bytecode(bytecode)
        data.y = torch.tensor([label], dtype=torch.float)
        data_list.append(data)
    except Exception as e:
        print(f"Error processing bytecode: {bytecode}")
        print(f"Error details: {e}")
        continue

# Save the data_list
torch.save(data_list, '/content/drive/MyDrive/Colab_Notebooks/Datasets/data_list.pt')


Processing Bytecodes: 530it [09:28,  1.25it/s]

Error processing bytecode: 0x60806040819052600080549091829173ffffffffffffffffffffffffffffffffffffffff169061029a346064020480156108fc029184818181858888f150503360009081526001602052604090205415925060c2915050573360008181526002602090815260408083205460019092529091205491935061170c916107d0606f909102044391909103020490508173ffffffffffffffffffffffffffffffffffffffff166108fc829081150290604051600060405180830381858888f150505050505b5050336000908152600260209081526040808320439055600190915290208054340190550000a165627a7a72305820eec7a3b2d41e2464f8850830914ddb8830d12b0afcf57c149c2cdb38048b2e800029
Error details: 


Processing Bytecodes: 2185it [32:14,  1.05it/s]

Error processing bytecode: 0x6080604052600436106100915760003560e01c806388d519c41161005957806388d519c4146101565780638da5cb5b146101815780639d1b464a146101c25780639e1a00aa146101ed578063f2fde38b1461024857610091565b80631cc2c911146100965780633f4ba83a146100e45780635c975abb146100fb57806383197ef0146101285780638456cb591461013f575b600080fd5b6100e2600480360360408110156100ac57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919080359060200190929190505050610299565b005b3480156100f057600080fd5b506100f9610523565b005b34801561010757600080fd5b506101106105dc565b60405180821515815260200191505060405180910390f35b34801561013457600080fd5b5061013d6105ef565b005b34801561014b57600080fd5b50610154610680565b005b34801561016257600080fd5b5061016b61073b565b6040518082815260200191505060405180910390f35b34801561018d57600080fd5b50610196610741565b604051808273ffffffffffffffffffffffffffffffffffffffff16815260200191505060405180910390f35b3480156101ce57600080fd5b506101d7610765565b604051808281526020019150506

Processing Bytecodes: 3234it [48:32,  2.32it/s]

...

Processing Bytecodes: 6224it [1:34:36,  1.41s/it]

Error processing bytecode: 0x6080604052600436106100695760003560e01c80635c60da1b116100435780635c60da1b146101135780639965b3d614610128578063f1739cae1461013d57610078565b8063025313a2146100805780630add8140146100be5780632463aba5146100d357610078565b366100785761007661017d565b005b61007661017d565b34801561008c57600080fd5b506100956101ad565b6040805173ffffffffffffffffffffffffffffffffffffffff9092168252519081900360200190f35b3480156100ca57600080fd5b506100956101d2565b3480156100df57600080fd5b50610076600480360360208110156100f657600080fd5b503573ffffffffffffffffffffffffffffffffffffffff166101f7565b34801561011f57600080fd5b506100956102a4565b34801561013457600080fd5b50610076610341565b34801561014957600080fd5b506100766004803603602081101561016057600080fd5b503573ffffffffffffffffffffffffffffffffffffffff16610466565b60006101876102a4565b905060405136600082376000803683855af43d6000833e8080156101a9573d83f35b3d83fd5b7f6279e8199720cf3557ecd8b58d667c8edc486bd1cf3ad59ea9ebdfcae0d0dfac5490565b7f8ddbac328deee8d986ec3a7b933a196f969

Processing Bytecodes: 8105it [2:00:57,  1.23s/it]

Error processing bytecode: 0x608060405234801561001057600080fd5b50600436106100ea5760003560e01c806383995b9a1161008c578063a457c2d711610066578063a457c2d714610261578063a9059cbb14610291578063bec57f05146102c1578063dd62ed3e146102dd576100ea565b806383995b9a146102095780638da5cb5b1461022557806395d89b4114610243576100ea565b806323b872dd116100c857806323b872dd1461015b578063313ce5671461018b57806339509351146101a957806370a08231146101d9576100ea565b806306fdde03146100ef578063095ea7b31461010d57806318160ddd1461013d575b600080fd5b6100f761030d565b60405161010491906113c7565b60405180910390f35b6101276004803603810190610122919061117e565b61039f565b60405161013491906113ac565b60405180910390f35b6101456103bd565b6040516101529190611529565b60405180910390f35b610175600480360381019061017091906110eb565b6103c7565b60405161018291906113ac565b60405180910390f35b6101936104c8565b6040516101a09190611544565b60405180910390f35b6101c360048036038101906101be919061117e565b6104d1565b6040516101d091906113ac565b60405180910390f35b6101f360048036038101906

Processing Bytecodes: 10571it [2:39:26,  1.10it/s]


In [ ]:
labels[530]

'0x60806040819052600080549091829173ffffffffffffffffffffffffffffffffffffffff169061029a346064020480156108fc029184818181858888f150503360009081526001602052604090205415925060c2915050573360008181526002602090815260408083205460019092529091205491935061170c916107d0606f909102044391909103020490508173ffffffffffffffffffffffffffffffffffffffff166108fc829081150290604051600060405180830381858888f150505050505b5050336000908152600260209081526040808320439055600190915290208054340190550000a165627a7a72305820eec7a3b2d41e2464f8850830914ddb8830d12b0afcf57c149c2cdb38048b2e800029'

## Extra: Different embedding technique using Node2Vec

In [ ]:
!pip install node2vec

In [ ]:
import torch
from evm_cfg_builder.cfg import CFG
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import json
from node2vec import Node2Vec

# Specify the path to your JSON file
json_file_path = "/content/drive/MyDrive/Colab_Notebooks/Datasets/reentrancy_train_dataset.json"

# Load the JSON data into a Python dictionary
with open(json_file_path, 'r') as json_file:
    dataset = json.load(json_file)

def process_block(block):
    block_opcode = [info[0] for info in block['info']]
    block_opcode = ' '.join(block_opcode)
    return block_opcode

def process_bytecode(bytecode):
    cfg = CFG(bytecode)
    sorted_data_mapping = {
        str(block): {
            'pos': block.start.pc,
            'info': [(instr.mnemonic, instr.description) for instr in block.instructions],
            'out': [str(out_block) for out_block in block.all_outgoing_basic_blocks],
        }
        for block in cfg.basic_blocks
    }
    sorted_keys = sorted(sorted_data_mapping.keys(), key=lambda key: sorted_data_mapping[key]['pos'])
    position_mapping = {block: idx for idx, block in enumerate(key for key in sorted_keys)}

    G = nx.DiGraph()

    for index, key in enumerate(position_mapping):
        G.add_node(index, attr=process_block(sorted_data_mapping[key]))
        for des in sorted_data_mapping[key]['out']:
            G.add_edge(index, position_mapping[des])

    node2vec = Node2Vec(G, dimensions=128)
    model = node2vec.fit(window=10, epochs=10)
    embedding = model.wv.vectors[0]

    return embedding


bytecodes = dataset['bytecode']
labels = dataset['is_reentrancy']

positive_embeddings = []
negative_embeddings = []

In [ ]:
for bytecode, label in tqdm(zip(bytecodes, labels), desc="Processing Bytecodes"):
    if bytecode == "0x":
        continue
    try:
        graph_embedding = process_bytecode(bytecode)
    except Exception as e:
        print(f"Error processing bytecode: {bytecode}")
        print(f"Error details: {e}")
        graph_embedding = np.zeros(128)  # generate zero vector

    if label == 1:
        positive_embeddings.append(graph_embedding)
    else:
        negative_embeddings.append(graph_embedding)

negative_embeddings = np.vstack(negative_embeddings)
positive_embeddings = np.vstack(positive_embeddings)

np.save('/content/drive/MyDrive/Colab_Notebooks/Datasets/reentrancy_negative_embeddings.npy', negative_embeddings)
np.save('/content/drive/MyDrive/Colab_Notebooks/Datasets/reetrancy_positive_embeddings.npy', positive_embeddings)

Output hidden; open in https://colab.research.google.com to view.